In [ ]:
# 衆院HPから議員データを取得

import csv
import requests
from lxml import html
import time
import random
import pandas as pd

# 入力と出力のCSVファイルパス
input_csv_path = 'Shugiin.csv'
output_csv_path = 'Shugiin_output.csv'

# 出力ファイルのヘッダー
output_header = ['kanji', 'kana', 'url', '議員名', 'かな', '選挙区', '会派', '略歴']

# CSV読み込みとデータ処理
with open(input_csv_path, 'r', encoding='cp932') as infile:
    reader = csv.DictReader(infile)

    # 空のデータフレームを作成
    df = pd.DataFrame(columns=output_header)

    for row in reader:
        url = row['url']

        try:
            # URLにアクセス
            response = requests.get(url)
            response.raise_for_status()
            tree = html.fromstring(response.content)

            # 議員名とかなを取得
            name_element = tree.xpath('//h2[@id="TopContents"]')
            name_text = name_element[0].text_content().strip() if name_element else "N/A"
            # 名前とかなの分割
            if '（' in name_text and '）' in name_text:
                name_parts = name_text.split('（')
                name_kanji = name_parts[0].strip()
                name_kana = f"（{name_parts[1].replace('）','').strip()}）"
            else:
                name_kanji = "N/A"
                name_kana = "N/A"

            # 選挙区と会派を取得
            district_party_element = tree.xpath('//h2[@id="TopContents"]/following-sibling::br[1]/following-sibling::text()[1]')
            district_party_text = district_party_element[0].strip() if district_party_element else "N/A"
            if '選出、' in district_party_text:
                district_party_parts = district_party_text.split('選出、')
                district = district_party_parts[0].strip()
                party = district_party_parts[1].strip()
            else:
                district = "N/A"
                party = "N/A"

            # 略歴を取得 - <br><br> 以降を選択
            bio_elements = tree.xpath('//h2[@id="TopContents"]/following-sibling::br[3]/following-sibling::text()')
            bio_text = ''.join([bio.strip() for bio in bio_elements]).strip() if bio_elements else "N/A"

            # 結果を辞書に格納
            result = {
                'kanji': row['kanji'],
                'kana': row['kana'],
                'url': url,
                '議員名': name_kanji,
                'かな': name_kana,
                '選挙区': district,
                '会派': party,
                '略歴': bio_text.replace('\n', '').replace('○', '。')
            }

            # デバッグ用出力
            print(f"Result for {url}: {result}")

            # データフレームに追加
            df = pd.concat([df, pd.DataFrame([result])], ignore_index=True)

            # スリープ（1～3秒のランダム）
            time.sleep(random.uniform(1, 3))

        except Exception as e:
            print(f"Error processing URL {url}: {e}")

# 結果をCSVに保存
df.to_csv(output_csv_path, encoding='cp932', index=False)

print("CSV出力が完了しました。")


In [1]:
# 参院ＨＰから略歴を取得

import csv
import requests
from lxml import html
import time
import random

# CSVファイルの読み込み
input_csv_path = 'Sangiin.csv'
output_csv_path = 'Sangiin_output.csv'

# 出力ファイルのヘッダー
output_header = ['num', 'kanji', 'kanji2', 'kana', 'kaiha', 'url', 'name', 'title', 'biography']

# CSV読み込みとデータ処理
with open(input_csv_path, 'r', encoding='cp932') as infile, open(output_csv_path, 'w', encoding='cp932', newline='') as outfile:
    reader = csv.DictReader(infile)
    writer = csv.DictWriter(outfile, fieldnames=output_header)
    writer.writeheader()

    for row in reader:
        url = row['url']

        try:
            # URLにアクセス
            response = requests.get(url)
            response.raise_for_status()
            tree = html.fromstring(response.content)

            # 氏名の抽出
            name = tree.xpath('//div/div/div/div/div/h1')
            name_text = name[0].text_content().strip() if name else "氏名が見つかりません"

            # title の抽出（//ddから3つのコラムを取得）
            titles = tree.xpath('//dd')
            titles_text = ' '.join([t.text_content().strip() for t in titles[:3]]) if titles else "タイトルが見つかりません"

            # 略歴の抽出
            biography = tree.xpath('//p')
            biographies = [b.text_content().strip() for b in biography] if biography else ["略歴が見つかりません"]
            biographies_text = ' '.join(biographies)

            # 結果の書き込み
            writer.writerow({
                'num': row['num'],
                'kanji': row['kanji'],
                'kanji2': row['kanji2'],
                'kana': row['kana'],
                'kaiha': row['kaiha'],
                'url': url,
                'name': name_text,
                'title': titles_text,
                'biography': biographies_text
            })

            # スリープ（1～3秒のランダム）
            time.sleep(random.uniform(1, 3))

        except Exception as e:
            print(f"Error processing URL {url}: {e}")

print("CSV出力が完了しました。")


CSV出力が完了しました。
